## Recommendation System
----

#### Loading Packages and Dependencies

In [81]:
import pandas as pd
import numpy as np
import nltk
import warnings
from rake_nltk import Rake
from textacy import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.simplefilter('ignore')
sns.set_style('whitegrid')

#### Preprocessing Dataset

In [82]:
dataset = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7') #Loading the dataset
dataset = dataset[['Title', 'Genre','Director', 'Writer', 'Actors', 'Plot']] #Selecting only the columns which contain text
dataset.dropna(inplace=True) #Dropping all null values 
dataset.set_index('Title', inplace=True) #Setting up index as title
for columns in dataset.columns:
    dataset[columns] = dataset[columns].str.lower() # converting entire dataframe to lowercase
    dataset[columns] = dataset[columns].apply(lambda x: preprocessing.remove.remove_punctuation(x)) # removing punctuations
    dataset[columns] = dataset[columns] + ' '

#### Getting Description Keywords

In [83]:
def get_word_dict(text):
    r=Rake()
    r.extract_keywords_from_text(text)
    return ' '.join(r.get_word_degrees().keys())

dataset['Description Keywords'] = dataset.sum(axis=1) #Concatenating entire dataset into one column 
dataset.drop(columns=dataset.columns[dataset.columns != 'Description Keywords'], inplace=True) # Dropping rest of the columns
dataset['Description Keywords'] = dataset['Description Keywords'].apply(lambda x: get_word_dict(x)) #Extracting the keywords

#### Creating Cosine Matrix

In [84]:
count_vect = CountVectorizer(ngram_range=(1,3),stop_words=nltk.corpus.stopwords.words('English'),lowercase=True) #creating BOW
count_mat = count_vect.fit_transform(dataset['Description Keywords'])
cosine_matrix = cosine_similarity(count_mat,count_mat)

#### Create Movie Recomendation

In [85]:
def Movie_recommendation(movie):
    try:
        movie_title = pd.Series(dataset.index)
        ind = movie_title[movie_title==movie.title()].index[0]
        print("you've Entered:",movie)
        print()
        print('Recommendation:')
        for movie_ind in list(pd.Series(cosine_matrix[ind]).sort_values(ascending=False).head(6).index[1:]):
            print(movie_title[movie_ind])
    except:
        print('Sorry, movie not found')

#### Generating Recommendation

In [99]:
Movie_recommendation('The Matrix')

you've Entered: The Matrix

Recommendation:
The Terminator
V for Vendetta
Memento
Terminator 2: Judgment Day
The Avengers
